In [ ]:
# 🎯 MODEL EVALUATION SCRIPT — MULTI MODEL

import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
import json
import yaml
import sys
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

print("=" * 80)
print("🎯 MODEL EVALUATION PIPELINE — MULTI MODEL + UNIQUE EXPERIMENTS")
print("=" * 80)

# -------------------------
# 1️⃣ Load Config
# -------------------------
print("\n📋 Loading configuration files...")

try:
    # Load pipeline config
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    
    # ✅ NEW: Load experiments config (for reference if needed)
    with open("experiments_config.yml", "r") as f:
        experiments_cfg = yaml.safe_load(f)
    
    print("✅ Configuration Loaded\n")
except Exception as e:
    print(f"❌ Error loading config: {e}")
    sys.exit(1)

# ✅ CHANGED: Parse comma-separated model types from Git variable
MODEL_TYPES_RAW = pipeline_cfg["models"]["enabled"]
MODEL_TYPES = [m.strip() for m in MODEL_TYPES_RAW.split(",")]  # Split and clean

print(f"📋 Models to evaluate: {MODEL_TYPES}\n")

UC_CATALOG = pipeline_cfg["models"]["catalog"]
UC_SCHEMA = pipeline_cfg["models"]["schema"]
BASE_NAME = pipeline_cfg["models"]["base_name"]
NAMING_FMT = pipeline_cfg["models"]["naming"]["format"]

EXPERIMENT_NAME = pipeline_cfg["experiment"]["name"]
ARTIFACT_PATH = pipeline_cfg["experiment"]["artifact_path"]

metrics_cfg = pipeline_cfg["metrics"]["classification"]
PRIMARY_METRIC = metrics_cfg["primary_metric"]
TRACKED_METRICS = metrics_cfg["tracked_metrics"]
DIRECTION = metrics_cfg["direction"]

EVALUATION_LOG_TABLE = pipeline_cfg["tables"]["evaluation_log"]

# -------------------------
# 2️⃣ Initialize MLflow & Spark
# -------------------------
try:
    spark = SparkSession.builder.appName("ModelEvaluationMultiModel").getOrCreate()
    mlflow.set_tracking_uri("databricks")
    client = MlflowClient()

    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        raise Exception(f"Experiment not found: {EXPERIMENT_NAME}")

    print("🔥 MLflow + Spark loaded successfully\n")
except Exception as e:
    print(f"❌ MLflow Init Failed: {e}")
    sys.exit(1)

# -------------------------
# TABLE SCHEMA
# -------------------------
def get_evaluation_table_schema():
    return StructType([
        StructField("timestamp", TimestampType(), True),
        StructField("run_id", StringType(), True),
        StructField("run_name", StringType(), True),
        StructField("model_type", StringType(), True),  # ✅ NEW: Track model type explicitly
        StructField("model_name", StringType(), True),
        StructField("primary_metric", StringType(), True),
        StructField("primary_metric_value", DoubleType(), True),
        StructField("all_metrics_json", StringType(), True),
        StructField("params_json", StringType(), True),
        StructField("model_uri", StringType(), True)
    ])

# -------------------------
# TABLE CREATION
# -------------------------
def ensure_table_exists():
    try:
        spark.sql(f"DESCRIBE TABLE {EVALUATION_LOG_TABLE}")
        print(f"✅ Table exists: {EVALUATION_LOG_TABLE}")
    except:
        print(f"📝 Creating new table: {EVALUATION_LOG_TABLE}")
        empty_df = spark.createDataFrame([], get_evaluation_table_schema())
        empty_df.write.format("delta").option("overwriteSchema", "true").saveAsTable(EVALUATION_LOG_TABLE)
        print(f"✅ Table created: {EVALUATION_LOG_TABLE}")

# -------------------------
# EXISTING RUN NAMES
# -------------------------
def get_existing_run_names():
    try:
        df = spark.sql(f"SELECT DISTINCT run_name FROM {EVALUATION_LOG_TABLE}")
        existing = set(r.run_name for r in df.collect())
        print(f"📊 Found {len(existing)} existing runs in evaluation table")
        return existing
    except:
        print("📊 No existing runs found (table is empty)")
        return set()

# -------------------------
# FETCH RUNS (MODEL-WISE)
# -------------------------
def get_runs_for_model(model_type):
    """
    Fetch runs for a specific model type from MLflow.
    Uses run_name pattern matching to filter by model_type.
    """
    order = f"metrics.{PRIMARY_METRIC} {'DESC' if DIRECTION=='maximize' else 'ASC'}"

    runs = client.search_runs(
        [experiment.experiment_id],
        order_by=[order],
        max_results=500
    )

    # 🔥 FILTER ONLY THIS MODEL TYPE
    # Training script creates run names like: credit_risk_random_forest_rf_baseline_balanced
    # So we check if model_type is in the run_name
    filtered_runs = [
        run for run in runs
        if model_type in (run.info.run_name or "")
    ]
    
    print(f"   🔍 Found {len(filtered_runs)} runs for {model_type}")
    return filtered_runs

# -------------------------
# MAIN EVALUATION LOOP
# -------------------------
def main():
    """
    Main evaluation logic:
    1. For each model type (from Git variable)
    2. Fetch all runs for that model
    3. Keep only best run per unique experiment name
    4. Log new runs to evaluation table
    """
    
    ensure_table_exists()
    existing_run_names = get_existing_run_names()

    all_records = []
    
    print("\n" + "="*80)
    print("🚀 STARTING EVALUATION FOR ALL MODEL TYPES")
    print("="*80)

    for model_type in MODEL_TYPES:

        print(f"\n📊 Evaluating model type: {model_type.upper()}")
        print("-" * 80)

        # Generate full model name for UC registry
        model_name = NAMING_FMT.format(
            catalog=UC_CATALOG,
            schema=UC_SCHEMA,
            base_name=BASE_NAME,
            model_type=model_type
        )
        
        print(f"   📦 Model Name: {model_name}")

        # Fetch all runs for this model type
        runs = get_runs_for_model(model_type)

        if not runs:
            print(f"   ⚠️  No runs found for {model_type}")
            continue

        # Dictionary to store best run per experiment name
        run_dict = {}

        for run in runs:
            run_name = run.info.run_name or "unnamed_run"

            # Skip if already logged
            if run_name in existing_run_names:
                continue

            # Get primary metric value
            pm_val = run.data.metrics.get(PRIMARY_METRIC)
            
            if pm_val is None:
                continue

            # Keep only best run per experiment name
            if run_name not in run_dict or (
                DIRECTION == "maximize" and pm_val > run_dict[run_name]["primary_metric"]
            ):
                run_dict[run_name] = {
                    "run_id": run.info.run_id,
                    "run_name": run_name,
                    "primary_metric": pm_val,
                    "all_metrics": {
                        m: run.data.metrics.get(m)
                        for m in TRACKED_METRICS
                        if m in run.data.metrics
                    },
                    "params": run.data.params,
                    "model_uri": f"runs:/{run.info.run_id}/{ARTIFACT_PATH}"
                }

        # Convert to records for DataFrame
        for r in run_dict.values():
            all_records.append({
                "timestamp": datetime.now(),
                "run_id": r["run_id"],
                "run_name": r["run_name"],
                "model_type": model_type,  # ✅ NEW: Add model type
                "model_name": model_name,
                "primary_metric": PRIMARY_METRIC,
                "primary_metric_value": float(r["primary_metric"]) if r["primary_metric"] else 0.0,
                "all_metrics_json": json.dumps(r["all_metrics"]),
                "params_json": json.dumps(r["params"]),
                "model_uri": r["model_uri"]
            })
        
        print(f"   ✅ Found {len(run_dict)} new unique experiments for {model_type}")

    # -------------------------
    # SAVE TO DELTA TABLE
    # -------------------------
    
    if not all_records:
        print("\n" + "="*80)
        print("✅ No new experiments to log (all runs already evaluated)")
        print("="*80)
        return

    print("\n" + "="*80)
    print(f"💾 SAVING {len(all_records)} NEW EVALUATIONS TO TABLE")
    print("="*80)

    df = pd.DataFrame(all_records)
    spark_df = spark.createDataFrame(df, schema=get_evaluation_table_schema())

    spark_df.write.format("delta").mode("append").saveAsTable(EVALUATION_LOG_TABLE)

    print(f"\n✅ Successfully logged {len(all_records)} new evaluated experiments")
    print(f"📊 Table: {EVALUATION_LOG_TABLE}")
    
    # Show summary by model type
    print("\n📋 Summary by Model Type:")
    print("-" * 80)
    for model_type in MODEL_TYPES:
        count = sum(1 for r in all_records if r["model_type"] == model_type)
        if count > 0:
            print(f"   {model_type}: {count} experiments")
    
    print("=" * 80)
    print("🎉 EVALUATION COMPLETED SUCCESSFULLY!")
    print("=" * 80)

if __name__ == "__main__":
    main()